In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
with open('train-en/left.txt') as fopen:
    left = fopen.read().split('\n')
    
with open('train-en/right.txt') as fopen:
    right = fopen.read().split('\n')

In [3]:
import malaya

/home/ubuntu/.local/lib/python3.8/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.3.0 and strictly below 2.5.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.6.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(
/home/ubuntu/.local/lib/python3.8/site-packages/tensorflow_addons/utils/resource_loader.py:72: UserWarning: You are currently using TensorFlow 2.6.0 and trying to load a custom op (custom_ops/seq2seq/_beam_search_ops.so).
TensorFlow Addons has compiled its custom ops again

In [7]:
from malaya.text.rules import rules_normalizer, rules_compound_normalizer
from malaya.text.normalization import _is_number_regex
from malaya.text.function import replace_punct
from collections import defaultdict
import re
import random
import json

In [5]:
PUNCTUATION = '!"#$%&\'()*+,./:;<=>?@[\]^_`{|}~'

def case_of(text):
    return (
        str.upper
        if text.isupper()
        else str.lower
        if text.islower()
        else str.title
        if text.istitle()
        else str
    )

def strip_punct(word):
    left = []
    right = []
    i = 0
    while i < len(word):
        if word[i] in PUNCTUATION:
            left.append(word[i])
            i += 1
        else:
            break
    i = len(word) - 1
    while i > 0:
        if word[i] in PUNCTUATION:
            right.append(word[i])
            i -= 1
        else:
            break
    left = ''.join(left)
    right = ''.join(right[::-1])
    if len(right):
        word_ = word[:-len(right)]
    else:
        word_ = word
    word_ = word_[len(left):]
    return left, right, word_

In [13]:
def replace_words_punct(left_word, right_word):
    left_left, left_right, left_word = strip_punct(left_word)
    right_left, right_right, right_word = strip_punct(right_word)
    return f'{left_left}{right_word}{left_right}'

def random_replace_alignment(left, right, alignment, min_replace = 5, max_replace = 15):
    splitted_left = left.split()
    splitted_right = right.split()
    
    selected_alignment = []
    for s in alignment:
        l = s[0]
        r = s[1]
        try:
            if _is_number_regex(splitted_left[l].replace(',', '').replace('.', '')) or _is_number_regex(splitted_right[r].replace(',', '').replace('.', '')):
                continue
            elif splitted_left[l].isupper() or splitted_right[r].isupper():
                continue
            elif splitted_left[l] == splitted_right[r]:
                continue
            elif splitted_left[r].lower() in ['the', 'a', 'an', 'it', 'is', 'are']:
                continue
            else:
                selected_alignment.append((l, r))
        except:
            pass
    
    try:
        count_replace = random.randint(min_replace, min(max_replace, len(selected_alignment)))
        selected = random.sample(selected_alignment, count_replace)
        for s in selected:
            splitted_left[s[0]] = replace_words_punct(splitted_left[s[0]], splitted_right[s[1]])

        return ' '.join(splitted_left)
    
    except:
        return ' '.join(splitted_left)

In [8]:
eflomal = malaya.alignment.en_ms.eflomal(preprocessing_func=replace_punct)

In [14]:
%%time

alignment = eflomal.align(left[7106:7107], right[7106:7107])['forward'][0]

CPU times: user 6.78 ms, sys: 20 ms, total: 26.8 ms
Wall time: 128 ms


In [15]:
random_replace_alignment(left[7106], right[7106], alignment)

'The goal is bagi make sure pihak people yang menerima itu land adalah mereka ones who benar-benar deserve it.'

In [16]:
fast_text = malaya.language_detection.fasttext()
fast_text.predict([left[7106], right[7106], 'saya suka'])

['eng', 'malay', 'malay']

In [17]:
import copy

In [22]:
consonants = 'bcdfghjklmnpqrstvwxyz'

def augment(left, right):
    
    alignment = eflomal.align([left], [right])['forward'][0]
    left = random_replace_alignment(left, right, alignment)
    
    return left

In [23]:
augment(left[7105], right[7105])

'Hung Ga Hung Ga, Hung Kuen, or Hung Ga Kuen is yang type of selatan Chinese bela arts associated with Chinese officer Wong Fei Hung who once menguasai and mengajar this art.'

In [24]:
from tqdm import tqdm
import traceback

def loop(rows):
    rows, _ = rows
    new_left, new_right, original = [], [], []
    for i in tqdm(range(len(rows))):
        left, right = rows[i][0], rows[i][1]
        if len(left.split()) > 100 or len(right.split()) > 100:
            continue
        langs = fast_text.predict([left, right])
        if langs[0] == 'malay':
            continue
        if langs[1] in ['eng', 'ind']:
            continue
        try:
            new_left_ = augment(left, right)
            if new_left_ != left:
                new_left.append(new_left_)
                new_right.append(right)
                original.append(left)
        except Exception as e:
            print(traceback.format_exc())
    return [[new_left, new_right]]

In [ ]:
r = loop((list(zip(left[2300000:2500000], right[2300000:2500000])),0))

 17%|██████████████▎                                                                      | 33542/200000 [1:55:51<6:00:17,  7.70it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 29%|████████████████████████▎                                                            | 57352/200000 [3:25:15<6:34:37,  6.02it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 40%|██████████████████████████████████▎                                            

In [ ]:
en, ms = [], []
for i in range(len(r)):
    print(i, len(r[i][0]))
    en.extend(r[i][0])
    ms.extend(r[i][1])

In [30]:
en[:10]

['"Hampir kesemua 130 pelajar mengalami simptom yang muntah dan sesak ini nafas belajar di kelas three or four.',
 'Pengangkutan is mudah tetapi sangat berkesan.',
 'Culinary (Kulinologi(r) is a mix between the disiplin ilmu food science dan kulinari seni.',
 'Indonesia sebenarnya mempunyai three potensial lokasi, namely Sentul, Palembang, dan Lombok.',
 'Liptovske Sliace Liptovske Sliace is a kampung dan kawasan located di the Ruzomberok district wilayah Zilina, Slovakia.',
 'Apa tidaknya, these kedua-dua popular selebriti are among the big-name air penghibur.',
 'I harap this ini not dijadikan big issue, "besar katanya.',
 'Both suspek used a metal bowl biasa used untuk cendol minuman to conceal yang estimated dadah bernilai RM900,000 before sending it ke proses courier company for delivery.',
 '"Bagaimanapun, itu ship berjaya to diri selepas berita mengenai the minyak spill was disebarkan melalui the massa media," he said.',
 'Namun, a few hari kemudian the dokter declined karena pe

In [31]:
ms[:10]

['"Hampir kesemua 130 pelajar yang mengalami simptom muntah dan sesak nafas ini belajar di kelas sama ada tingkat tiga atau empat.',
 'Pengangkutan adalah mudah tetapi sangat berkesan.',
 'Kulinologi(r) adalah campuran antara disiplin ilmu makanan dan seni kulinari.',
 'Indonesia sebenarnya mempunyai tiga lokasi potensial, yakni Sentul, Palembang, dan Lombok.',
 'Liptovske Sliace Liptovske Sliace merupakan sebuah kampung dan kawasan perbandaran yang terletak di daerah Ruzomberok, wilayah Zilina, Slovakia.',
 'Apa tidaknya, kedua-dua selebriti popular ini antara penghibur tanah air yang berprofil besar.',
 'Saya harap ini tak dijadikan isu besar," katanya.',
 'Kedua-dua suspek disyaki menggunakan mangkuk besi yang biasa digunakan untuk minuman cendol bagi menyembunyikan dadah yang dianggarkan bernilai RM900,000 sebelum menghantarnya ke syarikat kurier terba proses penghantaran.',
 '"Bagaimanapun kapal itu berjaya melarikan diri selepas berita mengenai pembuangan minyak disebarkan melalu

In [ ]:
import json

with open('augmented-en-ms-v3-part2.json', 'w') as fopen:
    json.dump({'en': en, 'ms': ms}, fopen)